In [2]:
# Import necessary libraries
import pandas as pd
import numpy as np
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, LabelEncoder
from sklearn.metrics import accuracy_score, classification_report
import joblib

# Load the dataset
data = pd.read_csv('/workspaces/CI_Project/loan_data.csv')  # Update path if needed

# Target column
target_column = 'loan_status'

# Encode categorical variables
label_encoders = {}
for column in data.select_dtypes(include=['object']).columns:
    if column != target_column:  # Avoid encoding the target
        le = LabelEncoder()
        data[column] = le.fit_transform(data[column])
        label_encoders[column] = le

# Split data into features (X) and target (y)
X = data.drop(columns=[target_column])
y = data[target_column]

# Reduce precision for space efficiency
X = X.astype(np.float32)

# Split into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)

# Standardize features
scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)

# Train the Random Forest model with reduced complexity
rf_model = RandomForestClassifier(n_estimators=50, max_depth=10, random_state=42)  # Reduced number of trees and max depth
rf_model.fit(X_train, y_train)

# Evaluate the model
y_pred = rf_model.predict(X_test)
print("Random Forest Accuracy:", accuracy_score(y_test, y_pred))
print("\nClassification Report:")
print(classification_report(y_test, y_pred))

# Save the trained model and scaler with compression
joblib.dump(rf_model, 'random_forest_model_compressed.pkl', compress=3)  # Compress with level 3
joblib.dump(scaler, 'scaler_compressed.pkl', compress=3)

# Function for making predictions
def predict_loan_status(input_data):
    """
    Predicts the loan status based on input features.
    
    Args:
        input_data (list or array-like): Feature values for a single instance.
        
    Returns:
        str: Predicted loan status.
    """
    # Load the saved model and scaler
    model = joblib.load('random_forest_model_compressed.pkl')
    scaler = joblib.load('scaler_compressed.pkl')
    
    # Preprocess input data
    input_data = np.array(input_data).reshape(1, -1)  # Reshape for a single prediction
    input_data = scaler.transform(input_data)  # Scale the data
    
    # Predict
    prediction = model.predict(input_data)
    return prediction[0]

# Example test input
example_input = [
    30,    # person_age
    1,     # person_gender (encoded)
    2,     # person_education (encoded)
    50000, # person_income
    5,     # person_emp_exp
    1,     # person_home_ownership (encoded)
    15000, # loan_amnt
    3,     # loan_intent (encoded)
    12.5,  # loan_int_rate
    0.3,   # loan_percent_income
    10,    # cb_person_cred_hist_length
    720,   # credit_score
    0      # previous_loan_defaults_on_file (encoded)
]

# Make prediction
predicted_status = predict_loan_status(example_input)
print("Predicted Loan Status:", predicted_status)


Random Forest Accuracy: 0.9188148148148149

Classification Report:
              precision    recall  f1-score   support

           0       0.93      0.97      0.95     10493
           1       0.89      0.73      0.80      3007

    accuracy                           0.92     13500
   macro avg       0.91      0.85      0.87     13500
weighted avg       0.92      0.92      0.92     13500

Predicted Loan Status: 0


/home/vscode/.local/lib/python3.11/site-packages/sklearn/base.py:493: UserWarning: X does not have valid feature names, but StandardScaler was fitted with feature names
  warnings.warn(
